In [1]:
import pandas as pd
import json
import re
import numpy as np
import sys

In [2]:
SPfull={'Properties':{
         'OnlyLowEnergySPheno' : True,
         'AddTreeLevelUnitarityLimits':True,
        'MINPAR':[[1,'LambdaIN']],
        'ParametersToSolveTadpoles':['mu2'],
        'BoundaryLowScaleInput':[['\[Lambda]','LambdaIN']],
        'ListDecayParticles':['Fu','Fe','Fd','hh'],
        'ListDecayParticles3B': [['Fu','"Fu.f90"'],['Fe','"Fe.f90"'],['Fd','"Fd.f90"']],
        'RenConditionsDecays':[
               ['dCosTW', '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
               ['dSinTW', '-dCosTW/Tan[ThetaW]'],
               ['dg2', '1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan[ThetaW]^2 + (2*PiVZVP*Tan[ThetaW])/MVZ^2)'],
               ['dg1', 'dg2*Tan[ThetaW]+g2*dSinTW/Cos[ThetaW]- dCosTW*g2*Tan[ThetaW]/Cos[ThetaW]']
             ],
        'DEFINITION': {'MatchingConditions':
                 [['v' ,'vSM'], 
                  ['Ye', 'YeSM'],
                  ['Yd', 'YdSM'],
                  ['Yu', 'YuSM'],
                  ['g1', 'g1SM'],
                  ['g2', 'g2SM'],
                  ['g3', 'g3SM']]},
        'DefaultInputValues' :['LambdaIN' , 0.27]
           },
    'Index':{'OnlyLowEnergySPheno':0,'MINPAR':1, 'ParametersToSolveTadpoles':2, 
                'BoundaryLowScaleInput':3, 'DEFINITION':4, 'ListDecayParticles':5, 
                'ListDecayParticles3B':6, 'DefaultInputValues':7, 
                'AddTreeLevelUnitarityLimits':8, 'RenConditionsDecays':9}   
   }
SP={'Properties':{
         'OnlyLowEnergySPheno' : True,
         'AddTreeLevelUnitarityLimits':True,
        'MINPAR':[],
        'ParametersToSolveTadpoles':[],
        'BoundaryLowScaleInput':[],
        'ListDecayParticles':[],
        'ListDecayParticles3B': [],
        'RenConditionsDecays':[
               ['dCosTW', '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
               ['dSinTW', '-dCosTW/Tan[ThetaW]'],
               ['dg2', '1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan[ThetaW]^2 + (2*PiVZVP*Tan[ThetaW])/MVZ^2)'],
               ['dg1', 'dg2*Tan[ThetaW]+g2*dSinTW/Cos[ThetaW]- dCosTW*g2*Tan[ThetaW]/Cos[ThetaW]']
             ],
        'DEFINITION': {'MatchingConditions':
                 []},
        'DefaultInputValues' :{}
           },
    'Index':{'OnlyLowEnergySPheno':0,'MINPAR':1, 'ParametersToSolveTadpoles':2, 
                'BoundaryLowScaleInput':3, 'DEFINITION':4, 'ListDecayParticles':5, 
                'ListDecayParticles3B':6, 'DefaultInputValues':7, 
                'AddTreeLevelUnitarityLimits':8, 'RenConditionsDecays':9}   
   }

## Local definitions

In [3]:
def get_tadpoles_and_bilinears(L,dimL,exclude=None):
    #Get tadpoles from Lagrangina excluding bilinear `exclude`
    tadpoles=list(L[(np.logical_and(L==1,dimL==2))].index)
    ctdpl=[] # Parameters to be calculated from tadpoles
    cbln =[] # Bilinear input paramater
    for t in tadpoles:
        bln=smd.loc['Coupling',t]
        if bln!=exclude:
            ctdpl.append( bln )
        else:
            cbln.append(bln)
    return ctdpl,cbln

def get_input_parameters(L,dimL):
    #TODO: Include Yukawas
    sci=[]
    scalarint=list(L[(np.logical_and(L==1,dimL>2))].index)
    for sc in scalarint:
        sci.append( smd.loc['Coupling',sc] )
    return sci

#TODO: Incluede Input Yukawas
def get_input_parameters_IN(sci,suffix='IN'):
    sciIN=[]
    BLS=[]
    for p in sci:
        rp=re.search('(\w+)',p)
        if rp:
            sciIN.append( rp.groups()[0]+suffix )

    if len(sci)==len(sciIN):
        return sciIN
    else:
        sys.exit('ERROR: Input parameter mismatch {}!={}'.format(sci,sciIN))
        
def get_BoundaryLowScaleInput(sci,sciIN):
    BLS=[]
    for i in range(len(sci)):
        BLS.append([sci[i],sciIN[i]])
    return BLS
def get_MINPAR(sciIN):
    MP=[]
    for i in range(len(sciIN)):
        MP.append([i+1,sciIN[i]])
    return MP

def get_smyukawas(smd,def_smyukawas=
                      ['Down-Yukawa-Coupling', 'Lepton-Yukawa-Coupling', 'Up-Yukawa-Coupling']):
    smyc=[]
    for y in def_smyukawas:
        smyc.append( smd.loc['Coupling',y])
    return smyc

def get_other_yukawas(smd,L,dimL,def_smyukawas=
                      ['Down-Yukawa-Coupling', 'Lepton-Yukawa-Coupling', 'Up-Yukawa-Coupling']):
    def_yukawas=list(L[(np.logical_and(L==2,dimL==3))].index)
    oyc=[]
    for y in def_yukawas:
        if y not in def_smyukawas:
            oyc.append( smd.loc['Coupling',y])
    return oyc

def get_gauge_couplings(smc,smcouplings=['Hypercharge-Coupling','Left-Coupling','Strong-Coupling']):
    cs=[]
    for c in smcouplings:
        cs.append( smc.loc['Description'][ smc.loc['Description']==c ].index[0] )
    
    return cs

def get_SM_MatchingConditions(smd,smc,
                              smcouplings=['Hypercharge-Coupling','Left-Coupling','Strong-Coupling'],
                              def_smyukawas=
                              ['Down-Yukawa-Coupling', 'Lepton-Yukawa-Coupling', 'Up-Yukawa-Coupling']):
    #VEV
    smvev=smd.loc['Coupling','EW-VEV']
    #Gauge couplings
    cs=get_gauge_couplings(smc,smcouplings)
    #SM Yukawas
    smyc=get_smyukawas(smd,def_smyukawas)

    lsmcpl=[]
    smcpl =[smvev]+smyc+cs
    for c in smcpl:
        lsmcpl.append( [c,c+'SM'])
    return lsmcpl

def get_decay_particles(DecayParticles   = ['Fu', 'Fe', 'Fd', 'hh'],
                    DecayParticles3B = ['Fu', 'Fe', 'Fd']):
    LDecayParticles3B=[]
    for p in DecayParticles3B:
        LDecayParticles3B.append( [p,'"{}.f90"'.format(p)])
    return DecayParticles,LDecayParticles3B


def get_sm_DefaultInputValues(smd,sci,sciIN):
    for i in range(len(sci)):
        if smd.loc['Coupling','SM Higgs Selfcouplings']==sci[i]:
            d={sciIN[i]:0.27}
        else:
            d={}
        return d
    
def to_math_list( l ):
    import re
    sl=str(l).replace('[','{' ).replace(
                      ']','}'     ).replace(
                       "'",""   ).replace(
                       r'\\','\\')
    sl=re.sub(r'\\{(\w+)}',r'\\[\1]',sl)
    return sl

In [4]:
SPdf=pd.DataFrame(SPfull)

In [5]:
SPdf

,Index,Properties
AddTreeLevelUnitarityLimits,8,True
BoundaryLowScaleInput,3,"[[\[Lambda], LambdaIN]]"
DEFINITION,4,"{'MatchingConditions': [['v', 'vSM'], ['Ye', '..."
DefaultInputValues,7,"[LambdaIN, 0.27]"
ListDecayParticles,5,"[Fu, Fe, Fd, hh]"
ListDecayParticles3B,6,"[[Fu, ""Fu.f90""], [Fe, ""Fe.f90""], [Fd, ""Fd.f90""]]"
MINPAR,1,"[[1, LambdaIN]]"
OnlyLowEnergySPheno,0,True
ParametersToSolveTadpoles,2,[mu2]
RenConditionsDecays,9,"[[dCosTW, 1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - ..."


In [6]:
f=open('sp.json','w')
json.dump(SP,f)
f.close()

In [7]:
f=open('fullparamenter.json','r')
fp=json.load(f)
f.close()

In [8]:
smd=pd.DataFrame( fp['smdict'] )

In [9]:
smd.loc['Lorentz']=smd.loc['Lorentz'].apply(lambda l: [] if type(l)!=list else l)

In [10]:
dimL=smd.loc['Lorentz'].apply(lambda l: len(l) )

In [11]:
L=smd.loc['Lorentz'].apply(lambda l: len( list(np.unique(l))) )

In [13]:
SP['Properties']['ParametersToSolveTadpoles']=get_tadpoles_and_bilinears(L,dimL)[0]
SP['Properties']['ParametersToSolveTadpoles']

['mu2']

In [15]:
sci=get_input_parameters(L,dimL)
sci

['\\[Lambda]']

In [19]:
sciIN=get_input_parameters_IN(sci,suffix='IN')
sciIN

['LambdaIN']

In [20]:
SP['Properties']['BoundaryLowScaleInput']=get_BoundaryLowScaleInput(sci,sciIN)
SP['Properties']['BoundaryLowScaleInput']

[['\\[Lambda]', 'LambdaIN']]

In [21]:
SP['Properties']['MINPAR']=get_MINPAR(sciIN)
SP['Properties']['MINPAR']

[[1, 'LambdaIN']]

In [22]:
smc=pd.DataFrame( fp['coupling'] )

In [24]:
get_other_yukawas(smd,L,dimL)

[]

In [25]:
get_SM_MatchingConditions(smd,smc)

[['v', 'vSM'],
 ['Yd', 'YdSM'],
 ['Ye', 'YeSM'],
 ['Yu', 'YuSM'],
 ['g1', 'g1SM'],
 ['g2', 'g2SM'],
 ['g3', 'g3SM']]

In [26]:
#V=smd.loc['Lorentz'].apply(lambda l: l if len( list(np.unique(l)))==1 else None ).dropna()

In [27]:
#V.to_dict()

In [28]:
DP,DP3B=get_decay_particles(DecayParticles   = ['Fu', 'Fe', 'Fd', 'hh'],
                    DecayParticles3B = ['Fu', 'Fe', 'Fd'])

In [29]:
SP['Properties']['DEFINITION']['MatchingConditions']=get_SM_MatchingConditions(smd,smc)
SP['Properties']['DEFINITION']['MatchingConditions']

[['v', 'vSM'],
 ['Yd', 'YdSM'],
 ['Ye', 'YeSM'],
 ['Yu', 'YuSM'],
 ['g1', 'g1SM'],
 ['g2', 'g2SM'],
 ['g3', 'g3SM']]

In [30]:
SP['Properties']['ListDecayParticles']=DP
SP['Properties']['ListDecayParticles3B']=DP3B

In [31]:
SP['Properties']['DefaultInputValues']['LambdaIN']=0.27

In [32]:
get_sm_DefaultInputValues(smd,sci,sciIN)

{'LambdaIN': 0.27}

In [33]:
SP['Properties']['DefaultInputValues'].update( 
                     get_sm_DefaultInputValues(smd,sci,sciIN) )

In [34]:
SP['Properties']['DefaultInputValues']

{'LambdaIN': 0.27}

In [35]:
SP

{'Index': {'AddTreeLevelUnitarityLimits': 8,
  'BoundaryLowScaleInput': 3,
  'DEFINITION': 4,
  'DefaultInputValues': 7,
  'ListDecayParticles': 5,
  'ListDecayParticles3B': 6,
  'MINPAR': 1,
  'OnlyLowEnergySPheno': 0,
  'ParametersToSolveTadpoles': 2,
  'RenConditionsDecays': 9},
 'Properties': {'AddTreeLevelUnitarityLimits': True,
  'BoundaryLowScaleInput': [['\\[Lambda]', 'LambdaIN']],
  'DEFINITION': {'MatchingConditions': [['v', 'vSM'],
    ['Yd', 'YdSM'],
    ['Ye', 'YeSM'],
    ['Yu', 'YuSM'],
    ['g1', 'g1SM'],
    ['g2', 'g2SM'],
    ['g3', 'g3SM']]},
  'DefaultInputValues': {'LambdaIN': 0.27},
  'ListDecayParticles': ['Fu', 'Fe', 'Fd', 'hh'],
  'ListDecayParticles3B': [['Fu', '"Fu.f90"'],
   ['Fe', '"Fe.f90"'],
   ['Fd', '"Fd.f90"']],
  'MINPAR': [[1, 'LambdaIN']],
  'OnlyLowEnergySPheno': True,
  'ParametersToSolveTadpoles': ['mu2'],
  'RenConditionsDecays': [['dCosTW',
    '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
   ['dSinTW', '-dCosTW/Tan[ThetaW]'],
   ['dg2

## Output file

In [36]:
SP=pd.DataFrame(SP)
SP=SP.sort_values('Index')

In [37]:
SMp=pd.read_json('SMp.json')

In [38]:
SMp['Properties']#.apply(lambda d: d.get('Lorent'))

AlphaS                         {'Description': 'Alpha Strong'}
Gf                         {'Description': 'Fermi's constant'}
ThetaW       {'Description': 'Weinberg-Angle', 'DependenceN...
Ud                 {'Description': 'Right-Down-Mixing-Matrix'}
Ue               {'Description': 'Right-Lepton-Mixing-Matrix'}
Uu                   {'Description': 'Right-Up-Mixing-Matrix'}
Vd                  {'Description': 'Left-Down-Mixing-Matrix'}
Ve                {'Description': 'Left-Lepton-Mixing-Matrix'}
Vu                    {'Description': 'Left-Up-Mixing-Matrix'}
Yd           {'Description': 'Down-Yukawa-Coupling', 'Depen...
Ye           {'Description': 'Lepton-Yukawa-Coupling', 'Dep...
Yu           {'Description': 'Up-Yukawa-Coupling', 'Depende...
ZW           {'Description': 'W Mixing Matrix', 'Dependence...
ZZ                   {'Description': 'Photon-Z Mixing Matrix'}
\[Lambda]    {'Description': 'SM Higgs Selfcouplings', 'Dep...
aEWinv       {'Description': 'inverse weak coupling con

In [39]:
dictentries=['DefaultInputValues']
f=open('sp.m','w')
for i in SP.index:
    if type(SP.loc[i,'Properties'])==bool:
        f.write('{} = {};\n\n'.format(i,SP.loc[i,'Properties']))
    elif type(SP.loc[i,'Properties'])==list:
        f.write('{} = {};\n\n'.format(i,to_math_list(SP.loc[i,'Properties'])  ))
    elif type(SP.loc[i,'Properties'])==dict:
        d=SP.loc[i,'Properties']
        if i in dictentries:
            dfv='{} = {{'.format(i)
            for k in d.keys():
                dfv=dfv+'{} -> {}'.format(k,0.27) 
            dfv=dfv+'};\n\n'
            f.write(dfv)
        else:
            for k in d.keys():
                if k  in ['MatchingConditions']:
                    f.write('{}[{}]={};\n\n'.format(i,k,to_math_list( SP.loc[i,'Properties'][k] )    
                          ))
f.close()                

In [40]:
cat sp.m

OnlyLowEnergySPheno = True;

MINPAR = {{1, LambdaIN}};

ParametersToSolveTadpoles = {mu2};

BoundaryLowScaleInput = {{\[Lambda], LambdaIN}};

DEFINITION[MatchingConditions]={{v, vSM}, {Yd, YdSM}, {Ye, YeSM}, {Yu, YuSM}, {g1, g1SM}, {g2, g2SM}, {g3, g3SM}};

ListDecayParticles = {Fu, Fe, Fd, hh};

ListDecayParticles3B = {{Fu, "Fu.f90"}, {Fe, "Fe.f90"}, {Fd, "Fd.f90"}};

DefaultInputValues = {LambdaIN -> 0.27};

AddTreeLevelUnitarityLimits = True;

RenConditionsDecays = {{dCosTW, 1/2*Cos{ThetaW} * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))}, {dSinTW, -dCosTW/Tan{ThetaW}}, {dg2, 1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan{ThetaW}^2 + (2*PiVZVP*Tan{ThetaW})/MVZ^2)}, {dg1, dg2*Tan{ThetaW}+g2*dSinTW/Cos{ThetaW}- dCosTW*g2*Tan{ThetaW}/Cos{ThetaW}}};

